<a href="https://colab.research.google.com/github/yasirabd/solver-society-job-data/blob/main/2_3_3_Ekstrak_job_description_(skill_indonesia).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Objective
Ekstrak job position, hasil ekstraksi adalah:
1. **Pendidikan**: kualifikasi pendidikan
2. **Skill**: keahlian yang dibutuhkan

Data input yang dibutuhkan:
1. jobstreet_clean_tahap3.csv
2. seluruh file .py pada folder hmmtagger

Data output yang dihasilkan: -

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re

In [3]:
jobstreet = pd.read_csv("/content/drive/My Drive/Data Loker/jobstreet_clean_tahap3.csv")
job_description = jobstreet['job_description'].str.lower()

job_description.head()

0    [sep][sep]usia max. 35 tahun[sep]pendidikan mi...
1    responsibilities:[sep]brief description of dut...
2    [sep]lingkup pekerjaan: [sep]menjaga account d...
3    [sep]candidate must possess at least diploma i...
4    [sep]lingkup pekerjaan: [sep]menjaga account d...
Name: job_description, dtype: object

In [4]:
# check null
job_description.isnull().sum()

0

In [5]:
test = job_description[0]
test.split('[sep]')

['',
 '',
 'usia max. 35 tahun',
 'pendidikan min. d3',
 'pengalaman min. 3 tahun sebagai senior staff atau spv finance dari perusahaan distributor atau sejenisnya',
 'mempunyai leadership yang kuat, bertanggung jawab & jujur',
 'memiliki pengelolaan dan analisa yg baik, teliti dan cekatan',
 'bersedia visit ke kantor cabang secara rutin',
 'penempatan head office cempaka mas jakarta pusat',
 '',
 'tugas & tanggung jawab :',
 '',
 'melakukan supervisi tim admin finance(a/r, retur, refund & a/p)',
 'melakukan kontrol aging a/r & limit a/r',
 'membuat schedule a/p payment',
 'melakukan kontrol terhadap retur/refund(cn/dn)',
 'melakukan follow up terkait program-program & point reward',
 'melakukan kordinasi dengan pihak terkait',
 'memaintance utang piutang terhadap dealer-dealer',
 '',
 '']

# Ekstrak dengan model NLP bahasa indonesia/multilanguage

## NLP tasks
- detect language: id/en
- tokenization
- remove stopwords
- stemming/lemmatization
- part-of-speech tagging
- select only noun

In [6]:
!pip install spacy-langdetect -q
!pip install spacy[lookups] -q
!python -m spacy download en_core_web_sm -q
# !pip install transformers -q
# !wget https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.id.300.bin.gz
# !gunzip cc.id.300.bin.gz

     |████████████████████████████████| 1.0MB 2.8MB/s 
     |████████████████████████████████| 28.0MB 143kB/s 
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')


In [7]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
nltk.download('stopwords')
from nltk.tokenize import word_tokenize
nltk.download('averaged_perceptron_tagger')

import spacy
from spacy_langdetect import LanguageDetector
from spacy.lang.id.stop_words import STOP_WORDS

from gensim.models.wrappers import FastText

# POS-tagging modules
# from tagger import MainTagger
# from tokenization import Tokenization

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [8]:
# remove token [sep]
# tes = ' '.join(test.split('[sep]')).strip()
# tes

In [9]:
# build vocab bahasa indonesia dari Fasttext
# model = FastText.load_fasttext_format('cc.id.300.bin')
# dict_vocab = model.wv.vocab
# vocab = list(model.wv.vocab)

Ambil data indonesia, untuk diexplore

In [10]:
lang_model = spacy.load('en_core_web_sm')
lang_model.add_pipe(LanguageDetector(), name="language_detector", last=True)

corpus_id = list()
corpus_en = list()
corpus_other = list()

for idx, desc in enumerate(job_description):
    # remove token [sep]
    d = ' '.join(desc.split('[sep]')).strip()

    # check language
    doc = lang_model(d)
    lang = doc._.language['language']

    if lang == 'id':
        corpus_id.append(desc)
    elif lang == 'en':
        corpus_en.append(desc)
    else:
        corpus_other.append(desc)

In [11]:
print("banyak corpus indonesia:", len(corpus_id))
print("banyak corpus inggris:", len(corpus_en))
print("banyak corpus lain:", len(corpus_other))

banyak corpus indonesia: 7280
banyak corpus inggris: 6748
banyak corpus lain: 8


Example

In [12]:
# lang_model = spacy.load('en_core_web_sm')
# lang_model.add_pipe(LanguageDetector(), name="language_detector", last=True)

# # initialize tagging model
# lexicon_path = "/content/drive/My Drive/Data Loker/Lexicon.trn"
# ngram_path = "/content/drive/My Drive/Data Loker/Ngram.trn"
# tagger = MainTagger(lexicon_path, ngram_path, 0, 3, 3, 0, 0, False, 0.2, 0, 500.0, 1)
# tokenize = Tokenization()

# doc = lang_model(tes)
# # detect languange
# lang = doc._.language['language']
# print(lang)

# nlp_id = spacy.blank('id')
# doc = nlp_id(tes)

# words_list = list()
# for word in doc:
#     # jika punctuation
#     if word.is_punct:
#         continue

#     # jika stop words
#     if word.is_stop:
#         continue

#     # jika out-of-vocabulary
#     if word.text not in vocab:
#         continue
    
#     # jika panjang kata <=1
#     if len(word.text) <=3:
#         continue

#     words_list.append(word.text)

# sentence = ' '.join(words_list).strip()
# output = tokenize.sentence_extraction(tokenize.cleaning(sentence))
# word_tagged = list()
# for o in output:
#     strtag = " ".join(tokenize.tokenisasi_kalimat(o)).strip()
#     word_tagged += [" ".join(tagger.taggingStr(strtag))]

# # collect list id dari tag Noun
# skills_id = list()
# for word in word_tagged[0].split():
#     # cek tag
#     if word.split('/')[1] == "NN":
#         skills_id.append(word.split('/')[0])

# pd.Series(skills_id).drop_duplicates().tolist()

In [13]:
# word_tagged

## Do the loop thing
lakukan iterasi pada data.

In [14]:
# lang_model = spacy.load('en_core_web_sm')
# lang_model.add_pipe(LanguageDetector(), name="language_detector", last=True)

# # initialize tagging model
# lexicon_path = "/content/drive/My Drive/Data Loker/Lexicon.trn"
# ngram_path = "/content/drive/My Drive/Data Loker/Ngram.trn"
# tagger = MainTagger(lexicon_path, ngram_path, 0, 3, 3, 0, 0, False, 0.2, 0, 500.0, 1)
# tokenize = Tokenization()

# nlp_id = spacy.blank('id')

# id_counter, en_counter = 0, 0
# result = list()
# for idx, desc in enumerate(job_description):
#     # remove token [sep]
#     d = ' '.join(desc.split('[sep]')).strip()

#     # check language
#     doc = lang_model(d)
#     lang = doc._.language['language']

#     if lang == 'en':
#         result.append(['en', desc])
#         en_counter += 1

#     elif lang == 'id':

#         doc = nlp_id(d)

#         words_list = list()
#         for word in doc:
#             # jika punctuation
#             if word.is_punct:
#                 continue

#             # jika stop words
#             if word.is_stop:
#                 continue

#             # jika out-of-vocabulary
#             if word.text not in vocab:
#                 continue
            
#             # jika panjang kata <=3
#             if len(word.text) <=3:
#                 continue

#             words_list.append(word.text)

#         sentence = ' '.join(words_list).strip()
#         output = tokenize.sentence_extraction(tokenize.cleaning(sentence))
#         word_tagged = list()
#         for o in output:
#             strtag = " ".join(tokenize.tokenisasi_kalimat(o)).strip()
#             word_tagged += [" ".join(tagger.taggingStr(strtag))]

#         # collect skill dari tag Noun
#         id_skills = list()
#         for word in word_tagged[0].split():
#             # cek tag
#             if word.split('/')[1] == "NN":
#                 id_skills.append(word.split('/')[0])
        
#         result.append(['id', id_skills])
#         id_counter += 1
    
#     else:
#         result.append(['other', desc])

#     if idx%1000 == 0:
#         print('sudah selesai..', idx)

In [15]:
# en_counter, id_counter, len(job_description) - en_counter - id_counter

> Hasil yang diberikan belum cukup baik, kita coba dengan cara kedua yaitu dengan memperhatikan pattern data.

# Explore data pattern
Coba cari skill pada sentence/kalimat dengan kata kunci.

Contoh sentence:
data -> menguasai microsoft office.

Kita cari kata kunci 'menguasai' pada sentence agar bisa mengambil skill 'microsoft office'.

- Total data: 7317

Kata kunci untuk mendapatkan skill berserta jumlah pada data.
- kualifikasi: 3518
- kemampuan: 1673
- mampu: 825
- mengerti: 120
- memiliki: 801
- menguasai: 111
- mahir: 12
- pengalaman: 120
- disukai: 1
- menyukai: 2
- mempunyai: 2
- requirement: 12
- qualification: 3
- required skill: 5
- berkeahlian: 1
- skill: 2
- membuat: 32 

In [16]:
counter = 0
for doc in corpus_id:
    # if 'kualifikasi' in doc:
    #     counter += 1
    if 'kemampuan' in doc:
        counter += 1
    elif 'mampu' in doc:
        counter += 1
    elif 'mengerti' in doc:
        counter += 1
    elif 'memiliki' in doc:
        counter += 1
    elif 'menguasai' in doc:
        counter += 1
    elif 'mahir' in doc:
        counter += 1
    elif 'pengalaman' in doc:
        counter += 1
    elif 'disukai' in doc:
        counter += 1
    elif 'menyukai' in doc:
        counter += 1
    elif 'mempunyai' in doc:
        counter += 1
    elif 'requirement' in doc:
        counter += 1
    elif 'qualification' in doc:
        counter += 1
    elif 'required skill' in doc:
        counter += 1
    elif 'berkeahlian' in doc:
        counter += 1
    elif 'skill' in doc:
        counter += 1
    elif 'membuat' in doc:
        counter += 1
    elif 'kompetensi' in doc:
        counter += 1
    # else:
    #     print(doc)
counter, len(corpus_id)

(7210, 7280)

In [17]:
# POS-tagging modules
from tagger import MainTagger
from tokenization import Tokenization

# initialize tagging model
lexicon_path = "/content/drive/My Drive/Data Loker/Lexicon.trn"
ngram_path = "/content/drive/My Drive/Data Loker/Ngram.trn"
tagger = MainTagger(lexicon_path, ngram_path, 0, 3, 3, 0, 0, False, 0.2, 0, 500.0, 1)
tokenize = Tokenization()

In [24]:
list_keywords = ['kemampuan', 'mampu', 'mengerti', 'memiliki', 'menguasai',
                 'mahir', 'pengalaman', 'disukai', 'menyukai', 'mempunyai',
                 'berkeahlian', 'requirement', 'qualification', 'required skill',
                 'skill', 'membuat', 'kompetensi', 'kompeten', 'operasional',
                 'punya', 'paham', 'bisa', 'diutamakan', 'menggunakan',
                 'bersedia', 'jago', 'pernah bekerja']

extract_sentence = list()
for job_desc in corpus_id:
    # split berdasarkan token [sep]
    split_desc = [data.strip() for data in job_desc.split('[sep]')]

    sentence_match = list()
    for item in split_desc:
        # jika list keywords terdapat pada sentence
        if any(x in item for x in list_keywords):
            
            # remove punctuation
            item = re.sub(r'[^\w\s]', '', item)
            # remove extra space
            item = re.sub(' +', ' ', item) 

            # tagging kata
            output = tokenize.sentence_extraction(tokenize.cleaning(item))
            word_tagged = list()
            for o in output:
                strtag = " ".join(tokenize.tokenisasi_kalimat(o)).strip()
                word_tagged += [" ".join(tagger.taggingStr(strtag))]
            
            # collect skill dari tag 'NN' (Noun) dan 'JJ' (Adjective)
            skill_id = list()
            for word in word_tagged[0].split():
                # cek tag
                word, tag = word.split('/')[0], word.split('/')[1] 
                if tag == "NN" or tag == "JJ":
                    skill_id.append(word)

            sentence_match.append(skill_id)
    
    if len(sentence_match) >= 1:
        # join list of skill
        s = list()
        for sentence in sentence_match:
            s+=sentence
        extract_sentence.append(s)
    else:
        extract_sentence.append(['tidak dicantumkan'])

In [25]:
extract_sentence[:3]

[['pengalaman',
  'min',
  'tahun',
  'senior',
  'staff',
  'spv',
  'finance',
  'perusahaan',
  'distributor',
  'sejenisnya',
  'leadership',
  'kuat',
  'jujur',
  'pengelolaan',
  'analisa',
  'baik',
  'teliti',
  'cekatan',
  'visit',
  'kantor',
  'cabang',
  'rutin',
  'schedule',
  'ap',
  'payment'],
 ['kemampuan',
  'negosiasi',
  'baik',
  'pekerjaan',
  'lapangan',
  'sepeda',
  'motor',
  'sim',
  'c'],
 ['kemampuan',
  'negosiasi',
  'baik',
  'pekerjaan',
  'lapangan',
  'sepeda',
  'motor',
  'sim',
  'c']]

# Build vocabulary pada skill id

In [26]:
# build vocab
vocab = list()

for skill in extract_sentence:
    vocab += skill

In [27]:
from collections import Counter

word_freq = dict(Counter(vocab))
word_freq.keys()

dict_keys(['pengalaman', 'min', 'tahun', 'senior', 'staff', 'spv', 'finance', 'perusahaan', 'distributor', 'sejenisnya', 'leadership', 'kuat', 'jujur', 'pengelolaan', 'analisa', 'baik', 'teliti', 'cekatan', 'visit', 'kantor', 'cabang', 'rutin', 'schedule', 'ap', 'payment', 'kemampuan', 'negosiasi', 'pekerjaan', 'lapangan', 'sepeda', 'motor', 'sim', 'c', 'apotekerdibutuhkan', 'apoteker', 'klinik', 'kecantikan', 'lumious', 'beauty', 'house', 'kualifikasiusia', 'maksimal', 'tahupendidikan', 'profesi', 'apotekerterbuka', 'fresh', 'graduatememiliki', 'stra', 'aktif', 'sipakomunikatif', 'team', 'pelayananpekerja', 'keras', 'sopan', 'disiplin', 'jawabbersedia', 'masa', 'training', 'deskripsi', 'pekerjaanbertanggung', 'persiapan', 'pelayanan', 'perawatan', 'klien', 'sop', 'informasi', 'rekomendasi', 'produk', 'kepuasan', 'klienasistensi', 'dokter', 'tindakanmempersiapkan', 'ruang', 'kebersihan', 'area', 'tindakan', 'asistensi', 'doktermelakukan', 'laporan', 'inventaris', 'stok', 'order', 'aia'

In [28]:
df_skill = pd.DataFrame(list(word_freq.items()), columns=['skill', 'frekuensi'])
df_skill.head()

,skill,frekuensi
0,pengalaman,6201
1,min,447
2,tahun,5757
3,senior,55
4,staff,234


In [29]:
df_skill.sort_values('frekuensi', ascending=False)

,skill,frekuensi
0,pengalaman,6201
2,tahun,5757
110,minimal,5162
15,baik,4348
87,bidang,4187
...,...,...
12243,lancarmemastikan,1
12244,piutangpiutang,1
12245,penyelesaiannyamemastikan,1
1737,eventpameran,1


In [30]:
# select frekuensi >= 10
df_skill = df_skill[df_skill.frekuensi >= 10].reset_index(drop=True)
df_skill = df_skill.sort_values('frekuensi', ascending=False).reset_index(drop=True)
df_skill

,skill,frekuensi
0,pengalaman,6201
1,tahun,5757
2,minimal,5162
3,baik,4348
4,bidang,4187
...,...,...
2464,batching,10
2465,beras,10
2466,cbersedia,10
2467,ungu,10


In [31]:
# df_skill.to_csv('skill_id_v2.csv', index=False)